# Tree viewer for node library

In [7]:
from ipytree import Tree, Node

In [218]:
from pathlib import Path

def list_nodes(node: Path):
    """
    Return a list of child directories and python files of a given Path' node'.
    Child directories and python files starting with '.' or '_' are excluded.
    
    Parameters:
    node (Path): A directory or a python file.
    
    Returns:
    nodes (List[Path]): List of child directories and python files. For python file 'node', list_pyiron_nodes(node) is called and the paths are added.
    """    
    node_path = node

    nodes = []
    if node.is_dir():
        for child in node_path.iterdir():
            if child.is_dir() and not child.name.startswith('.') and not child.name.startswith('_'):
                nodes.append(child)
    
        for child in node_path.glob('*.py'):
            if not child.name.startswith('.') and not child.name.startswith('_'):
                nodes.append(child) 
                
    elif node.is_file():
        for child in list_pyiron_nodes(node):
            nodes.append(child) 

        
    return nodes

In [230]:
from dataclasses import dataclass

@dataclass
class FunctionNode:
    name: str
    path: str

In [268]:
import ast

def list_pyiron_nodes(my_python_file):
    """
    This function reads a Python code file and looks for any assignments 
    to a list variable named 'nodes'. It then creates FunctionNode objects 
    for each element in this list and returns all FunctionNodes in a list.

    Params:
    ------
    my_python_file : str
        Path to the python file to be analysed

    Returns:
    -------
    nodes : list of FunctionNode
        List of FunctionNodes extracted from the Python file
    """    
    nodes = []
    with open(my_python_file, "r") as source:
        tree = ast.parse(source.read())
        
        for stmt in tree.body:
            if (isinstance(stmt, ast.Assign) and 
                len(stmt.targets) == 1 and 
                isinstance(stmt.targets[0], ast.Name) and 
                stmt.targets[0].id == 'nodes'):
        
                for n in stmt.value.elts:        
                    func_node = FunctionNode(name=n.id, 
                                             path=Path(my_python_file))
                    nodes.append(func_node)
                
    return nodes

In [235]:
def handle_click(event):
    """
    This function handles click events by adding nodes to the selected object
    if it does not already have any nodes. 

    Params:
    ------
    event : dict
        A dictionary representing the event object.

    Note:
    The event object should include the owner of the event (the object that was clicked),
    and the owner should have a 'nodes' property (a list of nodes) and a 'path' property (the path to the node).
    """    
    selected_node = event['owner']
    if (len(selected_node.nodes)) == 0:
        add_nodes(selected_node, selected_node.path)

In [254]:
def add_nodes(tree, parent_node):
    """
    This function adds child nodes to a parent node in a tree. It assumes the input 
    is a Abstract Syntax Tree (AST). It creates new nodes based on the attributes 
    of the parent node, updates icon style based on the type of node and finally 
    adds child nodes to the parent.

    Params:
    ------
    tree : ast
        The Abstract Syntax Tree

    parent_node : Node object
        The node of the AST to which child nodes must be added

    """
    
    for node in list_nodes(parent_node):
        name_lst = node.name.split('.')
        if len(name_lst) > 1:
            if 'py' == name_lst[-1]:
                node_tree = Node(name_lst[0])
                node_tree.icon = 'archive' # 'file'
                node_tree.icon_style = 'success'
            else:
                continue
        else:
            node_tree = Node(node.name)
            if isinstance(node, FunctionNode):
                node_tree.icon = 'codepen' # 'file-code' # 'code'
                node_tree.icon_style = 'danger'       
            else:    
                node_tree.icon = 'folder' # 'info', 'copy', 'archive' 
                node_tree.icon_style = 'warning'            
        
        node_tree.path = node
        tree.add_node(node_tree)
    
        node_tree.observe(handle_click, 'selected')
    

In [266]:
def view_tree(root_path='../pyiron_nodes/node_library'):
    """
    This function generates and returns a tree view of nodes starting from the 
    root_path directory.

    Params:
    ------
    root_path : str or Path, optional
        The root directory path from which the tree starts. 
        Defaults to '../pyiron_nodes/node_library'.

    Return:
    ------
    tree : Tree object
        A tree view object with nodes added to it.
    """
    import copy
    
    path = copy.copy(root_path)
    if isinstance(path, str):
        path = Path(root_path)

    tree = Tree(stripes=True)
    add_nodes(tree, parent_node=path)
    
    return tree

Note: available icons and type in ipytree
- style_values = ["warning", "danger", "success", "info", "default"]
- icons: https://fontawesome.com/v5/search?q=node&o=r (version 5) appears to work

In [267]:
view_tree('../pyiron_nodes/node_library')

Tree(nodes=(Node(icon_style='warning', name='databases'), Node(icon_style='warning', name='development'), Node…

In [45]:
def get_subpath_remove_ext(path, subpath_start):
    if subpath_start in path.parts:
        start_index = path.parts.index(subpath_start)
        subpath = Path(*path.parts[start_index:])
        subpath_no_ext = subpath.with_suffix('')
        return subpath_no_ext
    else:
        return "The subpath start was not found in the path."

In [12]:
def on_click(node):
     button.description = 'node' 

In [62]:
layout = widgets.Layout(width='auto', height='40px') #set width and height

button = widgets.Text(display='flex',
    flex_flow='column',
    align_items='stretch', 
    layout=layout)
button

Text(value='', layout=Layout(height='40px', width='auto'))

In [69]:
def on_click(node):
    path = get_subpath_remove_ext(node.path.path, 'node_library') / node.path.name
    path_str = str(path).replace('/', '.')
    button.value = str(path_str)

## Tree view widget connected to FlowWidget

In [1]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

In [2]:
from python.treeview import TreeView
import ipywidgets as widgets

from python.reactflow import PyironFlowWidget

In [3]:
from pyiron_workflow import Workflow   

Workflow.register("node_library.atomistic", "atomistic") 

wf = Workflow('compute_elastic_constants')
wf.engine = wf.create.atomistic.engine.ase.M3GNet()
wf.structure = wf.create.atomistic.structure.build.bulk('Pb', cubic=True)
wf.elastic = wf.create.atomistic.property.elastic.elastic_constants(structure=wf.structure, engine=wf.engine) #, parameters=parameters)
# out = elastic.pull()

In [4]:
fw = PyironFlowWidget(wf)
fw.react_flow_widget

ReactFlowWidget(edges='[{"source": "engine", "sourceHandle": "engine", "target": "elastic", "targetHandle": "e…

In [5]:
tree = TreeView('../pyiron_nodes/node_library', flow_widget=fw)
tree.tree

Tree(nodes=(Node(icon_style='warning', name='databases'), Node(icon_style='warning', name='development'), Node…